# NetCDF read/write
This notebook is focused on the [netCDF](https://www.unidata.ucar.edu/software/netcdf/) and how to read/write data from/in this format.    
The first module to work with netCDF was https://github.com/JuliaGeo/NetCDF.jl, but recently another module was developed by [@abarth](https://github.com/Alexander-Barth): [NCDatasets](https://github.com/Alexander-Barth/NCDatasets.jl). We will use it in this notebook.

In [1]:
using NCDatasets
if VERSION < v"0.7.0-beta.0"
    using Compat: @info, @warn, @debug
end
using Compat

## Read a netCDF
We will use the GEBCO bathymetry as an example.

In [2]:
ncfile = "data/gebco_30sec_16.nc"
if !isfile(ncfile)
    download("https://dox.ulg.ac.be/index.php/s/U0pqyXhcQrXjEUX/download", ncfile)
else
    @info("Bathymetry file already downloaded")
end

┌ Info: Bathymetry file already downloaded
└ @ Main In[2]:5


First we open the file for reading (**"r"** option).    

In [3]:
ds = Dataset(ncfile, "r");
@show ds;

ds = Dataset: data/gebco_30sec_16.nc
Group: /

Dimensions
   lat = 5402
   lon = 10800

Variables
  lat   (5402)
    Datatype:    Float64
    Dimensions:  lat
    Attributes:
     long_name            = Latitude
     standard_name        = latitude
     units                = degrees_north

  lon   (10800)
    Datatype:    Float64
    Dimensions:  lon
    Attributes:
     long_name            = Longitude
     standard_name        = longitude
     units                = degrees_east

  bat   (10800 × 5402)
    Datatype:    Float32
    Dimensions:  lon × lat
    Attributes:
     long_name            = elevation above sea level
     standard_name        = height
     units                = meters

Global attributes
  title                = GEBCO



We now know the variable name and attributes, as well as the dimensions.    
Let's read the longitude, latitude and bathymetry.

In [4]:
lon = ds["lon"];
lat = ds["lat"];
bathymetry = ds["bat"];

Note that with the previous commands, we load both the variable values and their attributes.

In [5]:
@show typeof(lon);
@show lon.attrib;

typeof(lon) = NCDatasets.CFVariable{Union{Missing, DateTime, AbstractCFDateTime, Number},1,NCDatasets.Variable{Float64,1},NCDatasets.Attributes}
lon.attrib =   long_name            = Longitude
  standard_name        = longitude
  units                = degrees_east



Then if we want to access the values only, we can use similar commands but with the `[:]` at the end:

In [6]:
bx = ds["lon"][:];
by = ds["lat"][:];
b = ds["bat"][:];

which yield different arrays, for instance:

In [7]:
@show typeof(bx);

typeof(bx) = Array{Union{Missing, Float64},1}


## Write a netCDF
### Time series
Let's start with a very simple example: a temperature time series (1-dimensional dataset), corresponding to the mean temperature in Uccle, Belgium (source: [IRM](https://www.meteo.be/meteo/view/fr/360955-Normales+mensuelles.html#ppt_5238240)).

In [8]:
temperatureUccle = [3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18., 14.9, 11.1, 6.8, 3.9];

First, let's create the file (clobber mode indicated by **"c"**).    
Note that we need to ensure the file doesn't already exist (otherwise we remove it).

In [9]:
outputfile = "temperature_series.nc"
if isfile(outputfile)
    rm(outputfile) 
    @info "Removing file $(outputfile)"
else
    @info "Creating a new netCDF"
end
ds = Dataset(outputfile, "c");

┌ Info: Removing file temperature_series.nc
└ @ Main In[9]:4


Then we create a time dimension. Its size should match that of the temperature vector.

In [10]:
defDim(ds, "time", length(temperatureUccle))

0

Now we can also create the *temperature* variable, which has *time* as dimension.

In [11]:
temperature = defVar(ds, "temperature", Float64, ("time",));

We will fill this variable with the values previously defined (note again the `[:]`):

In [12]:
temperature[:] = temperatureUccle;

and don't forget to close the `ds`:

In [13]:
close(ds);

If you have [`ncdump`](https://www.unidata.ucar.edu/software/netcdf/netcdf-4/newdocs/netcdf/ncdump.html) available on your machine, you can easily check the file content.     
Otherwise you can use the first part of the notebook, in which we've shown how to read a netCDF file.

In [14]:
run(`ncdump temperature_series.nc`)

netcdf temperature_series {
dimensions:
	time = 12 ;
variables:
	double temperature(time) ;
data:

 temperature = 3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18, 14.9, 11.1, 6.8, 3.9 ;
}


Process(`ncdump temperature_series.nc`, ProcessExited(0))

## Exercice
Create a netCDF file storing a 2D field (it can be a random field or real data).     
The procedure is similar to the previous example, except that we will work with longitude and latitude dimensions, instead of time.

The solution is available in [03-netCDF-2Dimensions.ipynb](./solutions/03-netCDF-2Dimensions.ipynb).